In [1]:
class Foo(object):
    def __init__(self, func):
        self._func = func

    def __call__(self):
        print ('class decorator runing')
        self._func()
        print ('class decorator ending')

@Foo
def bar():
    print ('bar')

bar()

class decorator runing
bar
class decorator ending


In [2]:
import webbrowser
webbrowser.open('http://www.baidu.com')

True

进行属性赋值上面有限制的数据结构。
这种情况下最好使用描述器

In [7]:
# 基类，使用描述器进行赋值
class Descriptor:
    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)
        
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

# 强制类型的描述器
class Typed(Descriptor):
    expected_type = type(None)
    
    # 复写父类的set方法
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('expected ' + str(self.expected_type))
        super().__set__(instance, value)

# 强制值范围的装饰器
class Unsigned(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expect >= 0')
        super().__set__(instance, value)


class MaxSized(Descriptor):
    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('Missing size option')
        super().__init__(name, **opts)
    
    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super().__set__(instance, value)
        

上面这些类就是要创建的数据模型或类型系统的基础构建模块。  
然后就可以根据类型系统实际定义各种不同的数据类型：

In [9]:
class Integer(Typed):
    expected_type = int
    
class UnsignedInteget(Integer, Unsigned):
    pass

class Float(Typed):
    expected_type = float

class UnsignedFloat(Float, Unsigned):
    pass

class String(Typed):
    expected_type = None

class SizedString(String, MaxSized):
    pass

当然也可以使用类装饰器来进行简化代码

In [10]:
# 利用类装饰器来进行属性赋值

def check_attributes(**kwargs):
    def decorate(cls):
        for key, value in kwargs.items():
            if isinstance(value, Descriptor):
                value.name = key
                setattr(cls, key, value)
            else:
                setattr(cls, key, value(key))
                
        return cls
    return decorate

@check_attributes(name=SizedString(size=8),shares=UnsignedInteget,price=UnsignedFloat)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

通常来讲，类装饰器是最灵活和好用的。首先他不依赖与任何其他的新技术，其次，装饰器可以很容易的添加或删除